In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory, RunnableLambda
from operator import itemgetter
from langchain_community.tools import TavilySearchResults
from langchain.tools import tool


In [17]:
VECTOR_STORE_DIR = os.path.join("..","..", "data", "vector_store")
embedding_model=OpenAIEmbeddings(model="text-embedding-ada-002")

# 벡터 스토어 불러오기 함수
def load_vector_store(store_name):
    store_path = os.path.join(VECTOR_STORE_DIR, store_name)
    
    # 폴더 존재 여부 확인
    if not os.path.exists(store_path) or not os.path.isdir(store_path):
        print(f"벡터 스토어 폴더가 존재하지 않습니다: {store_path}")
        return None
    
    # 데이터 존재 여부 확인
    if not os.listdir(store_path):
        print(f"벡터 스토어 폴더는 있지만 데이터가 없습니다: {store_path}")
        return None
    
    return Chroma(persist_directory=store_path,embedding_function=embedding_model)  

# 벡터 스토어 불러오기 
webtoon_vector_store = load_vector_store("webtoon_vector_store")
webnovel_vector_store = load_vector_store("webnovel_vector_store")


In [5]:
# OpenAI LLM 설정
llm = ChatOpenAI(model="gpt-4o", temperature=0.5)

# 사용자 질문 입력
query = input("질문을 입력하세요: ").strip()
if not query:
    print("유효한 질문을 입력하세요.")
    exit()

# 사용자 입력의 의도를 분류하는 프롬프트
query_type_prompt = PromptTemplate(
    input_variables=["query"],
    template="""
    사용자의 입력을 다음 5가지 중 하나로 분류하세요: 
    1-1. 웹툰 추천 

    1-2. 웹소설 추천

    1-3. 웹툰, 웹소설 함께 추천
    (예시)
    - "재밌는 무협 추천해줘" -> "다음과 같은 작품들을 추천드릴게요! (작품 추천) 웹툰과 웹소설 중 선택해 주시면 더 자세히 추천드릴 수 있습니다."
    - "로판 웹툰이나 웹소설 추천해줘" -> "다음과 같은 작품들을 추천드릴게요!"
    - "수요일에 볼게 없네." -> "수요일에 연재되는 웹툰이나 웹소설을 추천드릴게요!"
    - "요즘 20대 여자들은 뭐 봐?" -> "20대 여성분들이 많이 보고 있는 웹툰이나 웹소설을 추천드릴게요!"

    2-1. 일상 대화 -> 추천과 연관 지을 수 있음
    (예시)
    - "아 회사 가기 싫다." -> "출근은 언제나 힘들죠😭 출근길에 볼만한 코미디 일상물 웹툰을 추천드릴게요! (작품 추천)"
    - "어우 졸려." -> "잠을 확!!! 깨게 만드는 흥미진진한 웹툰을 추천드릴게요. (작품 추천)"
    - "햄버거 너무 맛있다." -> "맛있는 햄버거를 드셨나보군요! 부럽네요~🍔 먹음직스러운 음식이 나오는 웹툰 어떠세요~? (작품 추천)"
    - "아 주식 개망했다." -> "쉽지 않죠...ㅎㅎ 평범했던 주인공이 재벌 급으로 부자가 되는 웹툰을 추천드릴게요. 다시 의욕이 생길거에요!!"
    - "날씨 너무 좋다." -> "그쵸~ 날씨가 좋으면 기분도 덩달아 밝아지는 것 같아요😊"

    2-2. 일상 대화 -> 추천과 연관 지을 수 없음
    - "너는 진보야 보수야" -> "죄송합니다. 저는 정치적 견해를 가지고 있지 않습니다. 다른 질문을 주시면 웹툰, 웹소설을 추천해 드릴게요."
    - "20*30-10는 뭐야?" -> "590입니다. 웹툰, 웹소설과 관련된 질문을 주시면 추천해 드리겠습니다."

    
    **출력 형식 예시:**  
    올바른 출력: 1-1  
    잘못된 출력: "1-1", `웹툰 추천`, `"웹툰을 추천해 드릴게요!"`, `1-1. 웹툰 추천`

    반드시 `1-1, 1-2, 1-3, 2-1, 2-2` 중 하나만 출력하세요.  

    사용자 입력: {query}
    의도 번호:
    """
)


query_type_chain = LLMChain(llm=llm, prompt=query_type_prompt)
query_type = query_type_chain.run(query)


# 유효한 query_type 목록
valid_query_types = {"1-1", "1-2", "1-3", "2-1", "2-2"}

# query_type 검증
if query_type not in valid_query_types:
    print(f"경고: LLM이 예상치 못한 값 '{query_type}'을 반환했습니다.")
    query_type = "2-2"  # 기본값 설정 (일반 대화)

print(query_type)


C:\Users\Playdata\AppData\Local\Temp\ipykernel_36208\2693774078.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o", temperature=0.5)
C:\Users\Playdata\AppData\Local\Temp\ipykernel_36208\2693774078.py:51: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  query_type_chain = LLMChain(llm=llm, prompt=query_type_prompt)
C:\Users\Playdata\AppData\Local\Temp\ipykernel_36208\2693774078.py:52: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  query

1-3


In [13]:
# Retriever 설정 - 검색 설정
webtoon_retriever = webtoon_vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "score_threshold": 0.73,  
        "k": 5,  
    },
)

webnovel_retriever = webnovel_vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "score_threshold": 0.73,
        "k": 5,
    },
)

# webtoon db 검색 tool
@tool
def search_webtoon(query: str) -> list[Document]:
    """
    웹툰 검색
    """
    search_result = webtoon_retriever.invoke(query)

    print(f"🔹 검색된 결과 (search_webtoon) 개수: {len(search_result)}")

    if len(search_result) < 5:
        additional_results = search_web(query)  
        search_result.extend(additional_results)

    return search_result[:5]

# webnovel db 검색 tool
@tool
def search_webnovel(query: str) -> list[Document]:
    """
    웹소설 검색
    """
    search_result = webnovel_retriever.invoke(query)

    print(f"🔹 검색된 결과 (search_webtoon): {len(search_result)}")  
    
    if len(search_result) < 5:
        additional_results = search_web(query)  
        search_result.extend(additional_results)

    return search_result[:5]  

# web 검색 tool
@tool
def search_web(query: str) -> list[Document]:
    """
    실시간 웹 검색 (추가 추천용)
    """
    tavily_search = TavilySearchResults(max_results=2)
    search_result = tavily_search.invoke(query)

    print(f"🔹 검색된 결과 (search_web): {len(search_result)}")  

    return search_result if search_result else [Document(page_content="관련 검색 결과가 없습니다.")]

results=[]

# 의도별 tool 사용
if query_type == "1-1":
    print("search_webtoon(query) 실행")
    results = search_webtoon(query)

elif query_type == "1-2": 
    print("search_webnovel(query) 실행")
    results = search_webnovel(query)

elif query_type in ["1-3", "2-1"]: 
    print("search_webtoon(query) & search_webnovel(query) 실행")
    webtoon_results = search_webtoon(query)
    webnovel_results = search_webnovel(query)
    results = webtoon_results + webnovel_results

elif query_type == "2-2":
    print("일반 대화 처리 (LLM 실행됨)")
    final_response = llm.invoke(query)
    print(final_response)
    exit()

print(results)

search_webtoon(query) & search_webnovel(query) 실행
🔹 검색된 결과 (search_webtoon) 개수: 5
🔹 검색된 결과 (search_webtoon): 5
[Document(metadata={'age_rating': '전체이용가', 'episode': 238, 'genre': '로맨스', 'platform': '카카오웹툰', 'price': '-', 'status': '수 연재', 'title': '인소의 법칙', 'type': '웹툰', 'url': 'https://webtoon.kakao.com/content/%EC%9D%B8%EC%86%8C%EC%9D%98-%EB%B2%95%EC%B9%99/2334', 'views': 22293000}, page_content="인소의 법칙(카카오웹툰)-인터넷 소설 읽기가 취미였던 평범한 학생 함단이.\n 어느 날 자고 일어나니 옆집에는 아주 예쁜 여학생 반여령이,\n 학교에는 그 이름도 무시무시한 사대천왕이 나타났다!\n \n 하루아침에 소설처럼 뒤바뀐 세계,\n 그리고 함단이의 역할은 다름 아닌 여주인공 소꿉친구 중 하나.\n \n [인터넷 소설의 법칙!]\n 여주 친구는 각종 위험에 휩싸인다!, 장르:로맨스, 키워드: ['발랄한', '몰입되는', '로맨스', '차원이동물']"), Document(metadata={'age_rating': '전체연령가', 'episode': 20, 'genre': '일상', 'platform': '네이버 웹툰', 'price': '-', 'status': '20화 완결', 'title': '다섯작가의 이런저런이야기', 'type': '웹툰', 'url': 'https://comic.naver.com/webtoon/list?titleId=80536', 'views': '-'}, page_content="다섯작가의 이런저런이야기(네이버 웹툰)-본 컨텐츠는 GM대우 홍보용 웹툰입니다.\n조석, 김규삼, 네스티켓, 귀귀, 무적핑크! 대한민국을 대표하

In [12]:
# 추천 프롬프트
recommendation_prompt = PromptTemplate(
    input_variables=["query", "context"],
    template="""
    **역할**
    1. 너는 맨날 웹툰,웹소설을 보는 매니아야.
    2. 사용자의 요구와 성향을 잘 파악하고 적합한 작품을 추천해줘서 사용자도 매니아가 되게 해야해.
    3. 너의 추천이 잘못되면 사용자는 웹툰,웹소설에 아예 흥미를 잃어. 책임감을 가지고 추천해.

    사용자의 요구:
    {query}

    **검색된 정보**
    {context}

    **추천 가이드라인**
    - 사용자의 요청에 맞는 작품을 반드시 최소 5개 이상 추천하세요.
    - 항상 참조가능한 사실적 진술을 말합니다.
    - 사실만 말하며 자체적으로 정보를 추가하지 않습니다.
    - 작품을 추천할 때 제목과 간략한 설명을 포함하세요.
    - 장르를 정확하게 파악해. 
    (예시)
    로판 추천해줘 -> 장르:로판 혹은 로맨스판타지 인 것만 제공, 장르 : 로맨스 혹은 판타지인 것 제공 금지
    - 사용자가 보낸 query에서 키워드를 뽑아내고 적극 활용하세요.
    (예시)
    "짝사랑하는 남주가 나오는 로판이 보고 싶어." 라고 query가 들어왔다면 "짝사랑","짝사랑 남주"등의 키워드나 줄거리에 다음과 같은 단어가 나오는지 집중하세요.

    - 남주와 여주를 명확하게 구분하세요. 
    (예시)
    남주가 짝사랑하는 로판과 여주가 짝사랑하는 로판은 엄연히 다릅니다. 주인공의 성별을 잘 구별하세요.

    - 만약 사용자의 요청이 2-1 유형(일상 대화)이라면, 관련된 작품을 연결하여 자연스럽게 추천하세요.

    사용자 요청에 대한 추천을 생성하세요.
    """
)
recommendation_chain = LLMChain(llm=llm, prompt=recommendation_prompt)
context_str = "\n".join([doc.page_content for doc in results]) if results else "추천할 작품이 없습니다."

# 최종 응답 생성
recommendation = recommendation_chain.run(query=query, context=context_str)
final_response = f"{recommendation}"
print(final_response)

사용자님께서 삼각관계가 나오는 로맨스 판타지(로판)를 찾고 계시군요! 아래의 작품들을 추천드립니다:

1. **인소의 법칙** (카카오웹툰) - 평범한 학생 함단이가 소설 속 세계로 들어가 여주인공의 소꿉친구가 되어 겪는 이야기입니다. 삼각관계와 차원이동이 주요 소재로 등장합니다.

2. **도망친 곳이 낙원이었다 [19세 완전판]** (카카오페이지) - 주인공 세실리아가 회귀하여 여러 번의 결혼을 겪으며 삼각관계와 로맨스를 펼치는 이야기입니다. 회귀와 로맨스 판타지가 결합된 작품입니다.

현재 검색된 정보로는 삼각관계가 명확하게 드러나는 로맨스 판타지 작품이 제한적이지만, 위의 두 작품은 삼각관계와 관련된 요소를 포함하고 있어 추천드립니다. 추가로 더 많은 작품이 필요하시면 다른 플랫폼이나 검색을 통해 더 찾아보실 수 있습니다. 즐거운 독서 되세요!
